<a href="https://colab.research.google.com/github/akshaygrao77/Deep-learning-assignment1/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from keras.datasets import fashion_mnist
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import math
import wandb
from sklearn.model_selection import train_test_split
# Only used to show progressbar while inside an epoch
from tqdm.notebook import tqdm 

In [ ]:
!pip install wandb
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

wandb.init(project="cs21s002-ee21s113-dlassignment-1", entity="manu_data_analyst")

wandb: Currently logged in as: manu_data_analyst (use `wandb login --relogin` to force relogin)


# **Question 1 starts**

Loading dataset from fashio-mnist

In [5]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [22]:
def print_image_per_class():

        images = [Image.fromarray(image) for image in x_train]
        wandb.log({"examples": [wandb.Image(image) for image in images]})
      

In [23]:
print_image_per_class()


# **Question 2 starts**

In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
x_test.shape

(10000, 28, 28)

Flatten each image into a 1-d array to input to the network

In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

Normalise the array to [0,1] interval

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
x_train.shape

(60000, 784)

In [ ]:
x_test.shape

(10000, 784)

Categorical encoding for labels

In [ ]:
import tensorflow.keras as keras
num_categories = 10

y_train = keras.utils.to_categorical(y_train, num_categories)
y_test = keras.utils.to_categorical(y_test, num_categories)

In [ ]:
y_train.shape

(60000, 10)

In [ ]:
y_test.shape

(10000, 10)

In [ ]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [ ]:
class Activation:
    def computeSoftmaxGrad(self,H):
      grad_softmax_jacobian = np.zeros(H.shape[0]*H.shape[0]).reshape(H.shape[0],H.shape[0])
      for i in range(H.shape[0]):
        for j in range(H.shape[0]):
          if(i == j):
            grad_softmax_jacobian[i][j] = H[i]*(1-H[j])
          else:
            grad_softmax_jacobian[i][j] = -H[i]*H[j]
      return grad_softmax_jacobian
  
    def computeSigmoidGrad(self,H):
      # print("Grad Sigmoid ",H)
      # print("number of non-zeros in H:",np.count_nonzero(H))
      ret = H*(1-H)
      # print("Ret ",ret)
      # print("number of non-zeros in ret:",np.count_nonzero(ret))
      return ret 

    def computeTanhGrad(self,H):
      ret = 1 - H**2
      if(~(np.isfinite(ret).all())):
        if(np.isnan(ret).any()):
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("Gradient Tanh ret",ret)
        ret = replaceNanWithMaxAndMin(ret)
      return ret 
    
    def computeReluGrad(self,A):
       return 1.0 * (A > 0)

    def sigmoid(self, X):
      # print("Sigmoid",X)
      exp = np.exp(-X)
      # print("Exp:",exp)
      if(~(np.isfinite(exp).all())):
        if(np.isnan(exp).any()):
          print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
          print("Sigmoid X",X)
          print("Sigmoid exp before",exp)
        exp = replaceNanWithMaxAndMin(exp)
      
      ret = 1.0 / (1.0 + exp)
      # print("Ret sigmoid:",ret)
      return ret

    def relu(self,X):
      ret = np.maximum(0.0,X)
      return ret
    
    def tanh(self,X):
      ret = np.tanh(X)
      if(~(np.isfinite(ret).all())):
        if(np.isnan(ret).any()):
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("Tanh ret",ret)
        ret = replaceNanWithMaxAndMin(ret)
      return ret

    def softmax(self, X):
      Z = X-max(X)
      exps = np.exp(Z)
      if(~(np.isfinite(exps).all())):
        if(np.isnan(exps).any()):
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("softmax X",X)
          print("softmax exps",exps)
        exps = replaceNanWithMaxAndMin(exps)
      
      sum = np.sum(exps)
      if(~(np.isfinite(sum).all())):
        if(np.isnan(sum).any()):
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("softmax sum",sum)
        sum = replaceNanWithMaxAndMin(sum)

      ret = exps / sum
      if(~(np.isfinite(ret).all())):
        if(np.isnan(ret).any()):
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("softmax exps",exps)
          print("softmax sum",sum)
          print("softmax ret",ret)
        ret = replaceNanWithMaxAndMin(ret)

      if(~(np.isfinite(ret).all())):
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
          print("softmax X",X)
          print("softmax np.sum(exps)",np.sum(exps))
          print("softmax exps",exps)
      return ret

In [ ]:
  def replaceNanWithMaxAndMin(X):
    if(np.isscalar(X)):
      if(X == float("inf")):
          X = 1e+306
      elif(X == float("-inf")):
          X = 1e-306
      else:
        X = 1e-306
    else:
      if(~(np.isfinite(X).all())):
        nan_indices = np.argwhere(~(np.isfinite(X))).flatten()
        # print("nan_indices",nan_indices)
        for ind in nan_indices:
          # print("Ind",ind)
          # print("X[ind]",X[ind])
          if(X[ind] == float("inf")):
            X[ind] = 1e+306
          elif(X[ind] == float("-inf")):
            X[ind] = 1e-306
          else:
            X[ind] = 1e-306
    return X
      

In [ ]:
class InputLayer:
  number_of_inputs=0

  def __init__(self,number_of_inputs) -> None:
      self.number_of_inputs = number_of_inputs

In [ ]:
class Layer:
  number_of_neurons=0
  activation = 'sigmoid'
  # Weight intialization before layer to which current layer is defined(hence invalid on input layer)
  weight_initialization= None
  
  def __init__(self,number_of_neurons,activation) -> None:
      self.number_of_neurons = number_of_neurons
      self.activation = activation
      self.weight_initialization= None

In [ ]:
def xavierWeightInitialisation(num_previous_layer,num_next_layer): 
  # calculate the range for the weights
  lower, upper = -(math.sqrt(6.0) / math.sqrt(num_previous_layer + num_next_layer)), (math.sqrt(6.0) / math.sqrt(num_previous_layer + num_next_layer))
  # generate random numbers
  numbers = np.random.rand(num_previous_layer,num_next_layer)
  # scale to the desired range
  scaled = lower + numbers * (upper - lower)
  return scaled

def standardNormalInitialisation(num_previous_layer,num_next_layer): 
  return np.random.normal(size=(num_previous_layer,num_next_layer))

In [ ]:
class ANNModel:
  def __init__(self):
    self.layers = []

  def addInputLayer(self,input_layer_obj):
    self.input_layer_obj = input_layer_obj
  
  def addLayer(self,layer_obj):
    self.layers.append(layer_obj)
  
  def compile(self,loss,weight_init='stdnormal'):
    self.loss = loss
    self.randInitWeights(weight_init)
    self.randInitBiases()
    self.initAH()
    self.array_grad_L_by_A = [None]*(len(self.layers))
    self.array_grad_L_by_weight = [None]*(len(self.layers))
    self.array_grad_L_by_bias = [None]*(len(self.layers))
    self.array_grad_L_by_H = [None]*(len(self.layers))
    
  def initAH(self):
    self.A = []
    for l in range(len(self.layers)):
      self.A.append(np.zeros(self.layers[l].number_of_neurons))
    
    self.H = []
    self.H.append(np.zeros(self.input_layer_obj.number_of_inputs))
    for l in range(1,len(self.layers)+1):
      self.H.append(np.zeros(self.layers[l-1].number_of_neurons))
    
    # print(self.A)
    # print(self.H)

  def randWeightEachLayer(self,weight_init,num_previous_layer,num_next_layer):
    if(weight_init == 'stdnormal'):
      return standardNormalInitialisation(num_previous_layer,num_next_layer)
    elif(weight_init == 'xavier'):
      return xavierWeightInitialisation(num_previous_layer,num_next_layer)

  def randInitWeights(self,weight_init_overall):
    current_layer_weight_init = weight_init_overall
    if(not (self.layers[0].weight_initialization is None)):
      current_layer_weight_init = self.layers[0].weight_initialization
    self.weight_array=[self.randWeightEachLayer(current_layer_weight_init,self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons)]
    print("Weight init [0]:",current_layer_weight_init)
    
    for l in range(1,len(self.layers)):
      current_layer_weight_init = weight_init_overall
      if(not (self.layers[l].weight_initialization is None)):
        current_layer_weight_init = self.layers[l].weight_initialization
      
      print("Weight init [",l," ]:",current_layer_weight_init)
      self.weight_array.append(self.randWeightEachLayer(current_layer_weight_init,self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons))
      

  def randInitBiases(self):
    self.bias = []
    for l in range(len(self.layers)):
      self.bias.append(np.zeros(self.layers[l].number_of_neurons))
      # print(self.bias)
      # print(self.bias[l].shape)

  def predict(self,x_test):
    y_pred = [None]*x_test.shape[0]
    for index in range(x_test.shape[0]):
      y_pred[index] = self.feed_forward(x_test[index])
    y_pred = np.asarray(y_pred)
    return y_pred
  
  def initializeOptimizerStructures(self,optimizer='vanilla'):
    if(optimizer == 'momentum' or optimizer =='rmsprop'):
      self.prev_weight_update=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
      for l in range(1,len(self.layers)):
        self.prev_weight_update.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
      
      self.prev_bias_update = []
      for l in range(len(self.layers)):
       self.prev_bias_update.append(np.zeros(self.layers[l].number_of_neurons))
    elif(optimizer == 'nesterov'):
      self.prev_weight_update=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
      self.rollback_weight_update=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
      for l in range(1,len(self.layers)):
        self.prev_weight_update.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
        self.rollback_weight_update.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
      
      self.prev_bias_update = []
      self.rollback_bias_update = []
      for l in range(len(self.layers)):
       self.prev_bias_update.append(np.zeros(self.layers[l].number_of_neurons))
       self.rollback_bias_update.append(np.zeros(self.layers[l].number_of_neurons))
    
    elif(optimizer == 'adam' or optimizer == 'nadam'):
      self.m_w=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
      self.v_w=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
      self.m_w_hat=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
      self.v_w_hat=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
      for l in range(1,len(self.layers)):
        self.m_w.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
        self.v_w.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
        self.m_w_hat.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
        self.v_w_hat.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
      
      self.m_b = []
      self.v_b = []
      self.m_b_hat = []
      self.v_b_hat = []
      for l in range(len(self.layers)):
       self.m_b.append(np.zeros(self.layers[l].number_of_neurons))
       self.v_b.append(np.zeros(self.layers[l].number_of_neurons))
       self.m_b_hat.append(np.zeros(self.layers[l].number_of_neurons))
       self.v_b_hat.append(np.zeros(self.layers[l].number_of_neurons))
      
      self.update_count = 0
  
  def resetAccumulatedWeightsAndBiases(self):
    self.accumulated_array_grad_L_by_weight=[np.zeros((self.input_layer_obj.number_of_inputs,self.layers[0].number_of_neurons))]
    for l in range(1,len(self.layers)):
      self.accumulated_array_grad_L_by_weight.append(np.zeros((self.layers[l-1].number_of_neurons,self.layers[l].number_of_neurons)))
    self.accumulated_array_grad_L_by_bias = []
    for l in range(len(self.layers)):
      self.accumulated_array_grad_L_by_bias.append(np.random.rand(self.layers[l].number_of_neurons))
  
  # x_valid and y_valid is used only to just check test accuracy during each epoch. It is strictly not part of training
  def train(self,config,x_train,y_train,epochs,optimizer='vanilla',learning_rate=1,verbose=1,batch_size=10,l2_weight_decay=None,x_valid=None,y_valid=None):
    self.config = config
    if(l2_weight_decay is None):
      self.l2_weight_decay = 0
    else:
      self.l2_weight_decay = l2_weight_decay

    self.resetAccumulatedWeightsAndBiases()
    num_points_seen = 0
    self.initializeOptimizerStructures(optimizer)
    self.verbose=verbose
    if(self.verbose >= 4):
      self.print_state()
    for current_epoch in range(epochs):
      self.current_epoch = current_epoch
      if(self.verbose >=2):
        print("\n &&&&&&&&&&&&&&&&&&&&  Epoch ",current_epoch," &&&&&&&&&&&&&&&&&&&&&&&&")
      sum_of_loss = 0
      sum_of_loss_in_current_step = 0
      self.step_count = 0
      # for index in tqdm(range(10)):
      for index in tqdm(range(x_train.shape[0])):
        if(self.verbose >= 3):
          print("Data point number:",index)
        y_pred = self.feed_forward(x_train[index])
        self.back_prop(y_pred,y_train[index])
        if (optimizer == 'nesterov'):
          gamma = 0.9
          # First backup current parameters(to reset and continue training later)
          for l in range(len(self.layers)):
            self.rollback_weight_update[l] = self.weight_array[l]
            self.rollback_bias_update[l] = self.bias[l]
          # Set lookahead parameters into network
          for l in range(len(self.layers)):
            self.weight_array[l] = self.weight_array[l] - (gamma)*self.prev_weight_update[l]
            self.bias[l]=self.bias[l] - (gamma)*self.prev_bias_update[l]
          #Feed forward with lookahead parameters
          y_pred = self.feed_forward(x_train[index])
          #Back propogate to get the grad_L_byW_lookahead
          self.back_prop(y_pred,y_train[index])
          #Accumulate grad_L_by_W_lookahead
          for l in range(len(self.layers)):
            self.accumulated_array_grad_L_by_weight[l] = self.accumulated_array_grad_L_by_weight[l] + self.array_grad_L_by_weight[l]
            self.accumulated_array_grad_L_by_bias[l] = self.accumulated_array_grad_L_by_bias[l] + self.array_grad_L_by_bias[l]
          #Use previous backup to roll back parameters
          for l in range(len(self.layers)):
            self.weight_array[l] = self.rollback_weight_update[l]
            self.bias[l] = self.rollback_bias_update[l]
        else:
          for l in range(len(self.layers)):
            self.accumulated_array_grad_L_by_weight[l] = self.accumulated_array_grad_L_by_weight[l] + self.array_grad_L_by_weight[l]
            self.accumulated_array_grad_L_by_bias[l] = self.accumulated_array_grad_L_by_bias[l] + self.array_grad_L_by_bias[l]
        
        num_points_seen = num_points_seen + 1
        
        if(self.verbose >= 3):
          print("Predicted output:",y_pred)
          print("Actual output:",y_train[index])
        current_loss = computeLoss(y_pred,y_train[index],self.loss)
        sum_of_loss = sum_of_loss + current_loss
        sum_of_loss_in_current_step += current_loss
        if(self.verbose >= 4):
          self.print_state()
        if(self.verbose >= 3):
          print("Loss:",current_loss)
        
        if(num_points_seen % batch_size == 0 or num_points_seen == x_train.shape[0]):
          self.step_count += 1
          avg_loss_in_current_step = sum_of_loss_in_current_step/batch_size
          if(not(self.config is None)):
            wandb.log({"avg_loss_on_current_step": avg_loss_in_current_step,"step":self.step_count})
          self.updateParameters(optimizer,learning_rate)
          self.resetAccumulatedWeightsAndBiases()
          sum_of_loss_in_current_step = 0
      
      avg_loss = sum_of_loss/x_train.shape[0]
      if(not(self.config is None)):
        wandb.log({"avg_loss_on_current_epoch": avg_loss,"epoch":self.current_epoch})
      if(self.verbose >= 2):
        print("Average ",self.loss, " loss: ",avg_loss)
        self.showTrainAndTestAccuracyPostTraining(x_train,y_train,x_valid,y_valid)
  
  def showTrainAndTestAccuracyPostTraining(self,x_train,y_train,x_valid,y_valid):
    y_pred_train = self.predict(x_train)
    train_accuracy = calculateClassificationAccuracy(y_pred_train,y_train)
    print("Train accuracy ",train_accuracy)
    if(not(self.config is None)):
      wandb.log({"train_accuracy_on_current_epoch": train_accuracy,"epoch":self.current_epoch})
    if(not(x_valid is None or y_valid is None)):
      y_pred_test = self.predict(x_valid)
      test_accuracy = calculateClassificationAccuracy(y_pred_test,y_valid)
      print("Validation accuracy ",test_accuracy)
      if(not(self.config is None)):
        wandb.log({"valid_accuracy_on_current_epoch": test_accuracy,"epoch":self.current_epoch})

  def updateParameters(self,optimizer='vanilla',learning_rate=1):
    for l in range(len(self.layers)):
      allzeros = not np.any(self.accumulated_array_grad_L_by_weight[l])
      if(allzeros == True):
        # self.verbose=4.5
        print("Weight update zero %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        print("self.accumulated_array_grad_L_by_weight[",l,"]:",self.accumulated_array_grad_L_by_weight[l])

    
    if(optimizer == 'vanilla'):
      for l in range(len(self.layers)):
        self.weight_array[l] = self.weight_array[l] - (learning_rate)*self.accumulated_array_grad_L_by_weight[l]
        self.bias[l]=(self.bias[l] - (learning_rate)*self.accumulated_array_grad_L_by_bias[l])[0]
    
    elif(optimizer == 'nesterov'):
      gamma = 0.9

      for l in range(len(self.layers)):
        self.prev_weight_update[l] = gamma * self.prev_weight_update[l] + (learning_rate)* self.accumulated_array_grad_L_by_weight[l]
        self.prev_bias_update[l] = (gamma * self.prev_bias_update[l] + (learning_rate)* self.accumulated_array_grad_L_by_bias[l])[0]  
        self.weight_array[l] = self.weight_array[l] - self.prev_weight_update[l]
        self.bias[l]=self.bias[l] - self.prev_bias_update[l]
    
    elif(optimizer == 'momentum'):
      gamma = 0.9

      for l in range(len(self.layers)):
        self.prev_weight_update[l] = gamma * self.prev_weight_update[l] + (learning_rate)* (self.accumulated_array_grad_L_by_weight[l]+ 2 * self.l2_weight_decay * self.weight_array[l])
        self.prev_bias_update[l] = (gamma * self.prev_bias_update[l] + (learning_rate)* (self.accumulated_array_grad_L_by_bias[l] + 2 * self.l2_weight_decay * self.bias[l] ))[0]  
        self.weight_array[l] = self.weight_array[l] - self.prev_weight_update[l]
        self.bias[l]=self.bias[l] - self.prev_bias_update[l]
    
    elif(optimizer == 'rmsprop'):
      eps,beta1 = 1e-10,0.9

      for l in range(len(self.layers)):
        self.prev_weight_update[l] = beta1 * self.prev_weight_update[l] + (1 - beta1)* (self.accumulated_array_grad_L_by_weight[l] * self.accumulated_array_grad_L_by_weight[l])
        # print("self.prev_weight_update[l]",self.prev_weight_update[l])
        self.prev_bias_update[l] = (beta1 * self.prev_bias_update[l] + (1 - beta1)* (self.accumulated_array_grad_L_by_bias[l]*self.accumulated_array_grad_L_by_bias[l]))[0]  
        # print("self.prev_bias_update[l]",self.prev_bias_update[l])
        self.weight_array[l] = self.weight_array[l] - (learning_rate/np.sqrt(self.prev_weight_update[l] + eps)) * self.accumulated_array_grad_L_by_weight[l]
        # print("self.weight_array[l]",self.weight_array[l])
        self.bias[l] = self.bias[l] - ((learning_rate / np.sqrt(self.prev_bias_update[l] + eps)) * self.accumulated_array_grad_L_by_bias[l])[0]
        # print("self.bias[l]",self.bias[l])
        # print("self.bias[l] shape",self.bias[l].shape)
    
    elif(optimizer == 'adam'):
      eps,beta1,beta2 = 1e-8,0.9,0.999

      self.m_w[l] = beta1 * self.m_w[l] + (1 - beta1)* self.accumulated_array_grad_L_by_weight[l]
      self.m_b[l] = (beta1 * self.m_b[l] + (1 - beta1)* self.accumulated_array_grad_L_by_bias[l])[0]  

      self.v_w[l] = beta2 * self.v_w[l] + (1 - beta2)* (self.accumulated_array_grad_L_by_weight[l] * self.accumulated_array_grad_L_by_weight[l])
      self.v_b[l] = (beta2 * self.v_b[l] + (1 - beta2)* (self.accumulated_array_grad_L_by_bias[l]*self.accumulated_array_grad_L_by_bias[l]))[0]

      self.m_w_hat[l] = self.m_w[l]/(1- math.pow(beta1,self.update_count + 1))
      self.m_b_hat[l] = self.m_b[l] / (1-math.pow(beta1,self.update_count + 1))

      self.v_w_hat[l] = self.v_w[l]/(1- math.pow(beta2,self.update_count + 1))
      self.v_b_hat[l] = self.v_b[l] / (1-math.pow(beta2,self.update_count + 1))

      self.weight_array[l] = self.weight_array[l] - (learning_rate/np.sqrt(self.v_w_hat[l] + eps)) * self.m_w_hat[l]
      # print("self.weight_array[l]",self.weight_array[l])
      self.bias[l] = self.bias[l] - ((learning_rate / np.sqrt(self.v_b_hat[l] + eps)) * self.m_b_hat[l])[0]

      self.update_count = self.update_count + 1
    elif(optimizer == 'nadam'):
      eps,beta1,beta2 = 1e-8,0.9,0.999

      self.m_w[l] = beta1 * self.m_w[l] + (1 - beta1)* self.accumulated_array_grad_L_by_weight[l]
      self.m_b[l] = (beta1 * self.m_b[l] + (1 - beta1)* self.accumulated_array_grad_L_by_bias[l])[0]  

      self.v_w[l] = beta2 * self.v_w[l] + (1 - beta2)* (self.accumulated_array_grad_L_by_weight[l] * self.accumulated_array_grad_L_by_weight[l])
      self.v_b[l] = (beta2 * self.v_b[l] + (1 - beta2)* (self.accumulated_array_grad_L_by_bias[l]*self.accumulated_array_grad_L_by_bias[l]))[0]

      self.m_w_hat[l] = self.m_w[l]/(1- math.pow(beta1,self.update_count + 1))
      self.m_b_hat[l] = self.m_b[l] / (1-math.pow(beta1,self.update_count + 1))

      self.v_w_hat[l] = self.v_w[l]/(1- math.pow(beta2,self.update_count + 1))
      self.v_b_hat[l] = self.v_b[l] / (1-math.pow(beta2,self.update_count + 1))

      self.weight_array[l] = self.weight_array[l] - (learning_rate/np.sqrt(self.v_w_hat[l] + eps)) * (beta1*self.m_w_hat[l] + (((1-beta1)/(1-math.pow(beta1,self.update_count + 1)))* self.accumulated_array_grad_L_by_weight[l]))
      # print("self.weight_array[l]",self.weight_array[l])
      self.bias[l] = self.bias[l] - ((learning_rate / np.sqrt(self.v_b_hat[l] + eps)) * (beta1*self.m_b_hat[l] + (((1-beta1)/(1-math.pow(beta1,self.update_count + 1)))* self.accumulated_array_grad_L_by_bias[l])))[0]

      self.update_count = self.update_count + 1
    
    else:
      pass


  def feed_forward(self,inpX):
    self.H[0]= inpX
    for l in range(len(self.layers)):
      self.A[l]=np.matmul(self.H[l],self.weight_array[l]) + self.bias[l]
      self.H[l+1] = calculateActivation(self.layers[l].activation,self.A[l])
    return self.H[len(self.layers)]
  
  def back_prop(self,y_pred,y_train):
    if(self.verbose >= 5):
      print("Back prop starts ===========================")
    self.grad_L_by_ypred = self.computeLossGradient(y_pred,y_train,self.loss)
    self.grad_ypred_by_Aoutput = computeGradientHwrtA(y_pred,self.A[len(self.layers)-1],self.layers[len(self.layers)-1].activation)
    if(len(self.grad_ypred_by_Aoutput.shape)==2):
      self.array_grad_L_by_A[len(self.layers)-1]=np.matmul(self.grad_L_by_ypred,self.grad_ypred_by_Aoutput)
    else:
      self.array_grad_L_by_A[len(self.layers)-1]=np.multiply(self.grad_L_by_ypred,self.grad_ypred_by_Aoutput)
    
    for l in reversed(range(len(self.layers))):
      current_H=self.H[l]
      current_H=current_H[np.newaxis].T
      if(self.verbose >= 5):
        print("current_H:[",l,"]:",current_H.T)
        print("current_H shape:[",l,"]:",current_H.shape)
      current_grad_L_by_A = self.array_grad_L_by_A[l]
      if(len(current_grad_L_by_A.shape)==1):
        current_grad_L_by_A = current_grad_L_by_A[np.newaxis]
      
      if(self.verbose >= 5):
        print("current_grad_L_by_A[",l," ]:",current_grad_L_by_A)
        print("current_grad_L_by_A[",l," ] shape:",current_grad_L_by_A.shape)
      
      self.array_grad_L_by_weight[l]=np.matmul(current_H,current_grad_L_by_A)
      
      # allzeros = not np.any(self.array_grad_L_by_weight[l])
      # if(allzeros == True):
      #   # self.verbose=4.5
      #   print("Weight update zero **************************************************************************************")
      
      # if(self.verbose >= 5):
      #   print("self.array_grad_L_by_weight[",l,"]:",self.array_grad_L_by_weight[l])
      #   print("number of non-zeros in weight gradient:",np.count_nonzero(self.array_grad_L_by_weight[l]))

      
      self.array_grad_L_by_bias[l] = current_grad_L_by_A
      
      if(self.verbose >= 5):
        print("self.array_grad_L_by_bias[",l,"]:",self.array_grad_L_by_bias[l])
      
      self.array_grad_L_by_H[l] = np.matmul(self.weight_array[l],current_grad_L_by_A.T)
      
      # if(self.verbose >= 5):
      #   print("self.array_grad_L_by_H[",l,"]:",self.array_grad_L_by_H[l].T)
      
      if(l >= 1):
        current_grad_H_by_A = computeGradientHwrtA(self.H[l],self.A[l-1],self.layers[l-1].activation)
        current_grad_H_by_A = current_grad_H_by_A[np.newaxis]
        if(self.verbose >= 5):
          print("current_grad_H_by_A[",l,"]:",current_grad_H_by_A)
        self.array_grad_L_by_A[l-1] = np.multiply(self.array_grad_L_by_H[l].T,current_grad_H_by_A)
        if(self.verbose >= 5):
          print("self.array_grad_L_by_A[",(l-1),"]:",self.array_grad_L_by_A[l-1])

  def computeLossGradient(self,y_pred,y_actual,loss):
    grad_L_y = np.zeros(len(y_pred))
    if(loss=='categorical_crossentropy'):
      # This small epsilon is needed to avoid 1/0=inf
      epsilon = 1e-300
      # To hold label index from y_actual
      correct_index=findLabelIndex(y_actual)
      grad_L_y[correct_index] = -1/(np.maximum(y_pred[correct_index],epsilon))

      # if(not (self.l2_weight_decay is None)):
        

    return grad_L_y


  def print_state(self):
    for l in range(len(self.layers)):
      print("Layer",l)
      print("============================")
      print("Activation:",self.layers[l].activation)
      print("Number of units:",self.layers[l].number_of_neurons)
      print("Weights",self.weight_array[l])
      print("Biases",self.bias[l])
      # print("H's [ ",l," ]:",self.H[l])
      print("A's",self.A[l])
      print("**************************")

In [ ]:
def calculateClassificationAccuracy(y_pred,y_actual):
  total_samples = y_pred.shape[0]
  correctly_classified_count = 0.0
  for it in range(y_pred.shape[0]):
    classification_index = np.argmax(y_pred[it])
    if(y_actual[it][classification_index]== 1):
      correctly_classified_count= correctly_classified_count + 1
  accuracy = correctly_classified_count/total_samples
  print("Correctly classified:",correctly_classified_count)
  print("total_samples:",total_samples)
  return accuracy

In [ ]:
def calculateClassificationLoss(y_pred,y_actual):
  total_samples = y_pred.shape[0]
  sum_loss = 0.0
  for it in range(y_pred.shape[0]):
    current_loss = computeLoss(y_pred[it],y_actual[it],'categorical_crossentropy')
    sum_loss = sum_loss + current_loss
  avg_loss = sum_loss/total_samples
  return avg_loss

In [ ]:
def calculateActivation(activation,A):
  if(activation =='sigmoid'):
    return Activation().sigmoid(A)
  elif(activation == 'softmax'):
    return Activation().softmax(A)
  elif(activation == 'tanh'):
    return Activation().tanh(A)
  elif(activation == 'relu'):
    return Activation().relu(A)
  else:
    print("Invalid activation function")

def computeLoss(y_pred,y_actual,loss):
  loss_value=0
  if(loss=='categorical_crossentropy'):
    # This small epsilon is needed to avoid log(0)=undefined
    epsilon = 1e-300
    # To hold label index from y_actual
    correct_index=findLabelIndex(y_actual)
    loss_value = -math.log(np.maximum(y_pred[correct_index],epsilon),2)
    # loss_value = -np.log2(((y_pred[correct_index])+epsilon))
  return loss_value

def findLabelIndex(y_actual):
  for j in range(len(y_actual)):
    if(y_actual[j]==1):
      return j

def computeGradientHwrtA(H,A,activation):
  grad_H_by_A = []
  # Gradient is a kxk jacobian
  if(activation == 'softmax'):
    grad_H_by_A = Activation().computeSoftmaxGrad(H)
  elif(activation == 'sigmoid'):
    grad_H_by_A = Activation().computeSigmoidGrad(H)
  elif(activation == 'tanh'):
    grad_H_by_A = Activation().computeTanhGrad(H)
  elif(activation == 'relu'):
    grad_H_by_A = Activation().computeReluGrad(H)
  return grad_H_by_A


In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [ ]:
def runModelOnDataWithHP(config,x_train,x_valid,y_train,y_valid,activation_per_layer,epochs,hidden_layer_size,learning_rate,num_neuron_per_hidden_layer,optimizer,batch_size=128,l2_weight_decay=None):
  ann_model = ANNModel()
  input_layer = InputLayer(x_train.shape[1])
  ann_model.addInputLayer(input_layer)
  for num in range(hidden_layer_size):
    current_hidden_layer = Layer(num_neuron_per_hidden_layer,activation=activation_per_layer)
    ann_model.addLayer(current_hidden_layer)

  output_layer = Layer(10,activation='softmax')
  ann_model.addLayer(output_layer)

  ann_model.compile('categorical_crossentropy','xavier')

  ann_model.train(config,x_train=x_train,y_train=y_train,epochs=epochs,optimizer=optimizer,learning_rate=learning_rate,verbose=2,batch_size=batch_size,x_valid=x_valid,y_valid=y_valid,l2_weight_decay=l2_weight_decay)
  # ann_model.print_state()
  y_pred_train = ann_model.predict(x_train)
  train_accuracy = calculateClassificationAccuracy(y_pred_train,y_train)
  print("Train accuracy at end:",train_accuracy)
  if(not(config is None)):
    wandb.log({"train_accuracy": train_accuracy})
  avg_loss_train = calculateClassificationLoss(y_pred_train,y_train)
  print("Train avg_loss at end:",avg_loss_train)
  if(not(config is None)):
    wandb.log({"train_avg_loss": avg_loss_train})
  y_pred_valid = ann_model.predict(x_valid)
  validation_accuracy = calculateClassificationAccuracy(y_pred_valid,y_valid)
  print("Validation accuracy at end:",validation_accuracy)
  if(not(config is None)):
    wandb.log({"accuracy": validation_accuracy})   
  avg_loss_valid = calculateClassificationLoss(y_pred_valid,y_valid)
  print("Valid avg_loss at end:",avg_loss_valid)
  if(not(config is None)):
    wandb.log({"valid_avg_loss": avg_loss_valid})

In [ ]:
def hpTuningWithWandb(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        runModelOnDataWithHP(config,x_train=x_train,x_valid=x_valid,y_train=y_train,y_valid=y_valid,activation_per_layer=config.activation,epochs=config.epochs,hidden_layer_size=config.hidden_layer_size,learning_rate=config.learning_rate,num_neuron_per_hidden_layer=config.num_neuron_per_hidden_layer,optimizer=config.optimizer,batch_size=config.batch_size,l2_weight_decay=config.l2_weight_decay)

In [ ]:
# runModelOnDataWithHP(config=None,x_train=x_train,x_valid=x_valid,y_train=y_train,y_valid=y_valid,activation_per_layer='tanh',epochs=5,hidden_layer_size=3,learning_rate=0.001,num_neuron_per_hidden_layer=32,optimizer='momentum',batch_size=128,l2_weight_decay=0.005)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for pre_run_cell):


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for post_run_cell):


Exception: ignored

In [ ]:
# sweep_config = {
#   "method": "bayes",
#   "metric":{
#     "goal": "maximize",
#     "name": "accuracy"   
#   },
#   "parameters":{
#     "activation":{
#       "values":['sigmoid','tanh','relu']
#     },
#     "epochs":{
#       "values": [5,10]
#     },
#     "hidden_layer_size":{
#       "values":[3,5]
#     },
#     "learning_rate":{
#         "values":[0.001,0.0001]
#     },
#     "num_neuron_per_hidden_layer":{
#         "values":[32,64,128]
#     },
#     "optimizer":{
#         "values":['nesterov','momentum','rmsprop','adam']
#     }
#   }
# }

In [ ]:
dive_deep_sweep_config = {
  "method": "bayes",
  "metric":{
    "goal": "maximize",
    "name": "accuracy"   
  },
  "parameters":{
    "activation":{
      "values":['tanh','relu']
    },
    "epochs":{
      "values": [5,10]
    },
    "hidden_layer_size":{
      "values":[3,5]
    },
    "learning_rate":{
        "values":[0.001,0.0001]
    },
    "num_neuron_per_hidden_layer":{
        "values":[16,32,64,128]
    },
    "batch_size":{
        "values":[32,64,128,256]
    },
    "optimizer":{
        "values":['momentum']
    },
    "l2_weight_decay":{
        "values":[0.005,0.05,0.5]
    }
  }
}

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for pre_run_cell):


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for post_run_cell):


Exception: ignored

In [ ]:
import pprint
pprint.pprint(dive_deep_sweep_config)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for pre_run_cell):


Exception: ignored

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'activation': {'values': ['tanh', 'relu']},
                'batch_size': {'values': [32, 64, 128, 256]},
                'epochs': {'values': [5, 10]},
                'hidden_layer_size': {'values': [3, 5]},
                'l2_weight_decay': {'values': [0.005, 0.05, 0.5]},
                'learning_rate': {'values': [0.001, 0.0001]},
                'num_neuron_per_hidden_layer': {'values': [16, 32, 64, 128]},
                'optimizer': {'values': ['momentum']}}}
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for post_run_cell):


Exception: ignored

In [ ]:
sweep_id= wandb.sweep(dive_deep_sweep_config,project="Deep learning assignment1")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for pre_run_cell):


Exception: ignored

Create sweep with ID: v6fnhiph
Sweep URL: https://wandb.ai/akshaygrao/Deep%20learning%20assignment1/sweeps/v6fnhiph
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for post_run_cell):


Exception: ignored

In [ ]:
# sweep_id='617qaf79'

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for pre_run_cell):


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for post_run_cell):


Exception: ignored

In [ ]:
pprint.pprint(sweep_id)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for pre_run_cell):


Exception: ignored

'v6fnhiph'
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for post_run_cell):


Exception: ignored

In [ ]:
wandb.agent(sweep_id,hpTuningWithWandb)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fde0e316e90>> (for pre_run_cell):


Exception: ignored

wandb: Agent Starting Run: 5fhf9rpp with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 3
wandb: 	l2_weight_decay: 0.5
wandb: 	learning_rate: 0.0001
wandb: 	num_neuron_per_hidden_layer: 32
wandb: 	optimizer: momentum


Weight init [0]: xavier
Weight init [ 1  ]: xavier
Weight init [ 2  ]: xavier
Weight init [ 3  ]: xavier

 &&&&&&&&&&&&&&&&&&&&  Epoch  0  &&&&&&&&&&&&&&&&&&&&&&&&


  0%|          | 0/54000 [00:00<?, ?it/s]

...

  0%|          | 0/54000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Weight update zero %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
self.accumulated_array_grad_L_by_weight[ 4 ]: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Weight update zero %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
self.accumulated_array_grad_L_by_weight[ 0 ]: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Weight update zero %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
self.accumulated_array_grad_L_by_weight[ 1 ]: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [

wandb: Ctrl + C detected. Stopping sweep.


Weight update zero %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
self.accumulated_array_grad_L_by_weight[ 4 ]: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Weight update zero %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
self.accumulated_array_grad_L_by_weight[ 0 ]: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Weight update zero %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
self.accumulated_array_grad_L_by_weight[ 1 ]: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

Exception: ignored

In [ ]:
sample_xtrain=np.array([[1,2],[2,3],[3,4]])
sample_xtrain=sample_xtrain.astype('uint8')
print(sample_xtrain.shape)
print(sample_xtrain[0])

In [ ]:
sample_ytrain=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
sample_ytrain=sample_ytrain.astype('float32')
print(sample_ytrain)
print(sample_ytrain.shape)

In [ ]:
def runToyTest():
  ann_model = ANNModel()
  input_layer = InputLayer(2)
  hidden_layer1 = Layer(132,activation='sigmoid')
  hidden_layer2 = Layer(10,activation='sigmoid')
  output_layer = Layer(4,activation='softmax')

  ann_model.addInputLayer(input_layer)
  ann_model.addLayer(hidden_layer1)
  # ann_model.addLayer(hidden_layer2)
  ann_model.addLayer(output_layer)

  ann_model.compile('categorical_crossentropy')
  ann_model.train(sample_xtrain,sample_ytrain,1,'momentum',1,4)
  # ann_model.print_state()
  y_pred_test = ann_model.predict(sample_xtrain)
  print(y_pred_test)
  test_accuracy = calculateClassificationAccuracy(y_pred_test,sample_ytrain)
  print("Test Accuracy:",test_accuracy)

In [ ]:
# runToyTest()